In [7]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [2]:
spark = SparkSession \
        .builder \
        .appName("TopDomains") \
        .getOrCreate()

In [4]:
df=spark.read.format("csv").load("../data/w3data/nutch_URLs_0422/PRD_nutch_urls_20200422.txt")

In [1]:
df1=spark.read.format("csv").load("../data/w3data/nutch_URLs_0422/PRD_nutch1_urls_20200422.txt")

In [37]:
df.printSchema()

root
 |-- _c0: string (nullable = true)



In [5]:
df.count()

706251

In [2]:
df1.count()

112630

In [8]:
df=df.select(col("_c0").alias("url"))

In [69]:
df1=df1.select(col("_c0").alias("url"))

In [9]:
df.printSchema()

root
 |-- url: string (nullable = true)



In [10]:
from urllib.parse import urlparse

@udf
def getHost(url):
    #parsed_uri = urlparse('http://stackoverflow.com/questions/1234567/blah-blah-blah-blah' )
    #result = '{uri.scheme}://{uri.netloc}/'.format(uri=parsed_uri)
    return urlparse(url).hostname
    #return result

In [47]:
from urllib.parse import urlparse
import os.path

@udf
def getPathOne(url):
    #parsed_uri = urlparse('http://stackoverflow.com/questions/1234567/blah-blah-blah-blah' )
    #result = '{uri.scheme}://{uri.netloc}/'.format(uri=parsed_uri)
    #return urlparse(url).hostname
    #return result
    path = urlparse(url).path
    paths = path.split('/')
    if len(paths) > 1 :
        return paths[1]

In [52]:
from urllib.parse import urlparse
import os.path

@udf
def getPathTwo(url):
    #parsed_uri = urlparse('http://stackoverflow.com/questions/1234567/blah-blah-blah-blah' )
    #result = '{uri.scheme}://{uri.netloc}/'.format(uri=parsed_uri)
    #return urlparse(url).hostname
    #return result
    path = urlparse(url).path
    paths = path.split('/')
    if len(paths) > 2 :
        return paths[2]

In [11]:
@udf
def getHostId(url):
     return url.split(':')[0] 

In [12]:
@udf
def getProtocol(url):
    paths = url.split(':')
    if len(paths) > 1 :
        return paths[1].split('/')[0] 

In [13]:
@udf
def getPath1(url):
    paths = url.split(':')
    if len(paths) > 1 :
        paths2 = paths[1].split('/')
        if len(paths2) > 1:
            return paths2[1]

In [14]:
@udf
def getPath2(url):
    paths = url.split(':')
    if len(paths) > 1 :
        paths2 = paths[1].split('/')
        if len(paths2) > 2 :
            return paths2[2]

In [15]:
@udf
def getPath2(url):
    paths = url.split(':')
    if len(paths) > 1 :
        paths2 = paths[1].split('/')
        if len(paths2) > 2 :
            return paths2[2]

In [95]:
hostDf = df.withColumn('domain', getHost('url'))
hostDf = hostDf.withColumn('path1', getPathOne('url'))
hostDf = hostDf.withColumn('path2', getPathTwo('url'))

In [17]:
#hostDf = hostDf.withColumn('protocol', getProtocol('url'))

In [18]:
#hostDf = hostDf.withColumn('path1', getPath1('url'))

In [19]:
#hostDf = hostDf.withColumn('path2', getPath2('url'))

In [96]:
hostDf1 = df1.withColumn('domain', getHost('url'))
hostDf1 = hostDf1.withColumn('path1', getPathOne('url'))
hostDf1 = hostDf1.withColumn('path2', getPathTwo('url'))

In [97]:
hostDf.printSchema()

root
 |-- url: string (nullable = true)
 |-- domain: string (nullable = true)
 |-- path1: string (nullable = true)
 |-- path2: string (nullable = true)



In [98]:
hostDf1.printSchema()

root
 |-- url: string (nullable = true)
 |-- domain: string (nullable = true)
 |-- path1: string (nullable = true)
 |-- path2: string (nullable = true)



In [65]:
from pyspark.sql.functions import countDistinct
hostDf.select(countDistinct('domain')).show()


+----------------------+
|count(DISTINCT domain)|
+----------------------+
|                    40|
+----------------------+



In [194]:
hostDf.select('path1').show()

+--------------+
|         path1|
+--------------+
|          null|
|              |
|    activities|
|         appid|
|         appid|
|        impact|
|        impact|
|        impact|
|   initiatives|
|   initiatives|
|responsibility|
|responsibility|
|responsibility|
|responsibility|
|    activities|
|    activities|
|    activities|
|    activities|
|    activities|
|    activities|
+--------------+
only showing top 20 rows



In [108]:
hostnames = hostDf.select("domain", "path1") \
           .groupBy("domain", "path1")\
           .count()\
           .withColumnRenamed("count","nutch2count")\
           .orderBy("nutch2count", ascending=False)#\
           #.show(100, False)

In [109]:
hostnames1 = hostDf1.select("domain", "path1") \
           .groupBy("domain","path1")\
           .count()\
            .withColumnRenamed("count","nutch1count")\
           .orderBy("count", ascending=False)
           #.show(100, False)

In [110]:
hostnames.printSchema()

root
 |-- domain: string (nullable = true)
 |-- path1: string (nullable = true)
 |-- nutch2count: long (nullable = false)



In [113]:
nutch1v2 = hostnames1.join(hostnames, on=['domain', 'path1'], how='outer')\
                 .orderBy("nutch2count", ascending=False)

In [114]:
nutch1v2.show(100, False)

+-----------------------------------+-----------------------+-----------+-----------+
|domain                             |path1                  |nutch1count|nutch2count|
+-----------------------------------+-----------------------+-----------+-----------+
|www.ibm.com                        |blogs                  |40370      |228736     |
|developer.ibm.com                  |answers                |null       |160757     |
|procure.sby1.ibm.com               |whk                    |1906       |86669      |
|w3-03.ibm.com                      |services               |361        |78741      |
|d01db034.pok.ibm.com               |q_dir                  |37139      |73962      |
|w3-01.ibm.com                      |hr                     |1157       |10137      |
|productpages.w3ibm.mybluemix.net   |ProductPages           |204        |6511       |
|incentivesworkplace.atlanta.ibm.com|node                   |1          |5223       |
|secure.video.ibm.com               |channel          

In [104]:
nutch1v2_2 = hostnames1.join(hostnames,  how='outer', on=['domain','path1'])\
                 .orderBy("nutch2count", ascending=False)

AnalysisException: 'USING column `path1` cannot be resolved on the left side of the join. The left-side columns: [domain, nutch1count];'

In [204]:
hostnames = hostDf.select("domain",  "path1", "path2") \
           .where(col("domain") == "com.ibm.www") \
           .groupBy("path1")\
           .count()\
           .orderBy("count", ascending=False)\
           .show()

+-----+-------+
|path1|  count|
+-----+-------+
|blogs|3178314|
|  ibm|   2279|
|cloud|   1674|
|demos|     18|
+-----+-------+



In [193]:
hostnames.show(200, False)


+-----+-----+
|path1|count|
+-----+-----+
+-----+-----+

